DATA PREPROCESSING

In [1]:
import pandas as pd
import numpy as np
dataset= pd.read_csv('Titanic-Dataset.csv')
dataset.drop(dataset.columns[[3,8,10]],axis=1,inplace=True)

In [2]:
dataset.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [3]:
dataset.shape

(891, 9)

In [4]:
dataset.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
dtype: int64

In [5]:
x=dataset.iloc[:,2:9].values
y=dataset.iloc[:,1].values

In [6]:
print(x)

[[3 'male' 22.0 ... 0 7.25 'S']
 [1 'female' 38.0 ... 0 71.2833 'C']
 [3 'female' 26.0 ... 0 7.925 'S']
 ...
 [3 'female' nan ... 2 23.45 'S']
 [1 'male' 26.0 ... 0 30.0 'C']
 [3 'male' 32.0 ... 0 7.75 'Q']]


MISSING DATA

In [7]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values=np.NaN, strategy='median')
imputer.fit(x[:,2].reshape(891,1))
x[:,2]=imputer.transform(x[:,2].reshape(891,1)).reshape(1,891)

In [8]:
from sklearn.impute import SimpleImputer
imputer1= SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer1.fit(x[:,-1].reshape(891,1))
x[:,-1]= imputer1.transform(x[:,-1].reshape(891,1)).reshape(1,891)

In [9]:
x=np.array(x)

SPLITTING DATASET

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2)

In [11]:
print(x_train[0,:])

[3 'female' 27.0 0 0 7.925 'S']


ENCODING CATEGORICAL DATA

In [12]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
x_train[:,1]=le.fit_transform(x_train[:,1])
x_test[:,1]=le.fit_transform(x_test[:,1])

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[-1])],remainder='passthrough')
x_train=ct.fit_transform(x_train)
x_test=ct.fit_transform(x_test)

In [14]:
x=np.array(x)

In [15]:
print(x_train[1,:])

[0.0 0.0 1.0 1 0 62.0 0 0 80.0]


In [16]:
print(len(x_train[:,5]),len(x_test[:,5]))

712 179


FEATURE SCALING

In [17]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train[:,5]=sc.fit_transform(x_train[:,5].reshape(712,1)).reshape(1,712)
x_test[:,5]=sc.transform(x_test[:,5].reshape(179,1)).reshape(1,179)

In [18]:
from sklearn.preprocessing import StandardScaler
sc1=StandardScaler()
x_train[:,-1]=sc1.fit_transform(x_train[:,-1].reshape(712,1)).reshape(1,712)
x_test[:,-1]=sc1.transform(x_test[:,-1].reshape(179,1)).reshape(1,179)

In [19]:
from sklearn.preprocessing import StandardScaler
sc1=StandardScaler()
x_train[:,3]=sc1.fit_transform(x_train[:,3].reshape(712,1)).reshape(1,712)
x_test[:,3]=sc1.transform(x_test[:,3].reshape(179,1)).reshape(1,179)

In [20]:
print(x_train[0,:])

[0.0 0.0 1.0 0.8302298225337378 0 -0.18943688447351445 0 0
 -0.5284037201836289]


MODEL SELECTION (LOGISTIC REGRESSION)


In [21]:
from sklearn.linear_model import LogisticRegression
classifier= LogisticRegression(random_state=42)

In [22]:
from sklearn.model_selection import cross_val_score,cross_val_predict
cross= cross_val_score(estimator=classifier, X=x_train, y=y_train,cv=15)
print('accuracy=', cross.mean())

accuracy= 0.7921394799054374


In [23]:
classifier.fit(x_train,y_train)

LogisticRegression(random_state=42)

In [24]:
y_pred= classifier.predict(x_test)

In [25]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.8156424581005587


MODEL SELECTION (RANDOM FOREST CLASSIFICATION)

In [26]:
from sklearn.ensemble import RandomForestClassifier
classifier_1= RandomForestClassifier(n_estimators=10,random_state=42)

In [27]:
from sklearn.model_selection import cross_val_score
cross= cross_val_score(estimator=classifier_1, X=x_train, y=y_train,cv=5)
print('accuracy=', cross.mean())

accuracy= 0.789421845759874


In [28]:
from sklearn.model_selection import GridSearchCV
parameters={ 'n_estimators' : [10,20,30,40,60,100,80],
             'criterion' : ['gini', 'entropy', 'log_loss'],
             'max_depth' : [2,4,6,8,None],
             'max_features' : ['sqrt', 'log2', None],
             'class_weight' : ['balanced', 'balanced_subsample',None]}
grid_search= GridSearchCV( estimator = classifier_1,
                           param_grid= parameters,
                           scoring= 'accuracy',
                           cv= 3)
grid_search.fit(x_train,y_train)
best_accuracy= grid_search.best_score_
best_parameter= grid_search.best_params_
print(best_accuracy)
print(best_parameter)

0.8258695883416657
{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 100}


In [29]:
classifier_1.fit(x_train,y_train)

RandomForestClassifier(n_estimators=10, random_state=42)

In [30]:
y_pred_1=classifier_1.predict(x_test)

In [31]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred_1))

0.8324022346368715


In [32]:
from sklearn.ensemble import RandomForestClassifier
classifier_2= RandomForestClassifier(n_estimators=30,random_state=42,criterion='entropy', max_depth= 6, max_features='sqrt')
classifier_2.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=6, n_estimators=30,
                       random_state=42)

In [33]:
y_pred_2=classifier_2.predict(x_test)

In [34]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred_2))

0.8659217877094972
